## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# from config import g_key
import os
from dotenv import load_dotenv
load_dotenv()

weather_api_key = os.getenv("WEATHER_API_KEY")
g_key = os.getenv("G_KEY")

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Ushuaia,-54.8000,-68.3000,31.66,74,40,19.57,AR,shower snow
1,1,Maragogi,-9.0122,-35.2225,76.93,81,70,12.03,BR,broken clouds
2,2,Punta Arenas,-53.1500,-70.9167,37.42,75,20,25.32,CL,few clouds
3,3,Namibe,-15.1961,12.1522,63.73,87,11,4.14,AO,few clouds
4,4,Long Branch,40.3043,-73.9924,99.23,59,75,9.22,US,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like at your destination? ")) 
max_temp = float(input("What is the maximum temperature you would like at your destination? ")) 

What is the minimum temperature you would like at your destination? 60
What is the maximum temperature you would like at your destination? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Namibe,-15.1961,12.1522,63.73,87,11,4.14,AO,few clouds
5,5,Santa Fe,-31.6333,-60.7000,66.00,74,92,11.01,AR,moderate rain
13,13,Altay,47.8667,88.1167,61.63,59,100,4.97,CN,overcast clouds
22,22,Gayny,60.3071,54.3241,60.76,81,38,4.23,RU,scattered clouds
26,26,Bambous Virieux,-20.3428,57.7575,70.05,78,75,13.80,MU,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                7
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
# Checks for empty rows
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Namibe,AO,63.73,few clouds,-15.1961,12.1522,
5,Santa Fe,AR,66.00,moderate rain,-31.6333,-60.7000,
13,Altay,CN,61.63,overcast clouds,47.8667,88.1167,
22,Gayny,RU,60.76,scattered clouds,60.3071,54.3241,
26,Bambous Virieux,MU,70.05,light rain,-20.3428,57.7575,
27,Prieska,ZA,60.24,clear sky,-29.6641,22.7474,
32,Geraldton,AU,60.93,scattered clouds,-28.7667,114.6000,
39,Asmara,ER,61.95,broken clouds,15.3333,38.9333,
41,Novaya Ladoga,RU,64.89,clear sky,60.1025,32.3019,
45,Zhovtneve,UA,70.39,clear sky,50.6637,24.2492,


In [8]:
# Test block to ensure all elements of the code should work to grab hotel information based on coordinates 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": ""
}

lat = "54.9928"
lng = "82.4311"

params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make requests and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "-15.1961, 12.1522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [10]:
# Check that hotels were added to the dataframe
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Namibe,AO,63.73,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
5,Santa Fe,AR,66.00,moderate rain,-31.6333,-60.7000,Conquistador Hotel
13,Altay,CN,61.63,overcast clouds,47.8667,88.1167,Jinqiao Hotel
22,Gayny,RU,60.76,scattered clouds,60.3071,54.3241,
26,Bambous Virieux,MU,70.05,light rain,-20.3428,57.7575,Casa Tia Villa
27,Prieska,ZA,60.24,clear sky,-29.6641,22.7474,Avenue GuestHouse
32,Geraldton,AU,60.93,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
39,Asmara,ER,61.95,broken clouds,15.3333,38.9333,Hamasien Hotel
41,Novaya Ladoga,RU,64.89,clear sky,60.1025,32.3019,База Отдыха Горизонт
45,Zhovtneve,UA,70.39,clear sky,50.6637,24.2492,


In [11]:
hotel_df.loc[(hotel_df["Hotel Name"] == "")]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
22,Gayny,RU,60.76,scattered clouds,60.3071,54.3241,
45,Zhovtneve,UA,70.39,clear sky,50.6637,24.2492,
61,Sibut,CF,73.42,overcast clouds,5.7180,19.0739,
70,Beloha,MG,64.69,clear sky,-25.1667,45.0500,
80,Chik,RU,64.15,light rain,54.9928,82.4311,
111,Blagoyevo,RU,67.50,broken clouds,63.3667,47.9167,
118,Rochegda,RU,68.67,overcast clouds,62.8000,43.2500,
139,Chala,TZ,62.80,overcast clouds,-7.5833,31.2667,
141,Senno,RU,63.27,clear sky,59.5336,33.9167,
218,Novomalorossiyskaya,RU,65.43,few clouds,45.6348,39.8948,


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Namibe,AO,63.73,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
5,Santa Fe,AR,66.00,moderate rain,-31.6333,-60.7000,Conquistador Hotel
13,Altay,CN,61.63,overcast clouds,47.8667,88.1167,Jinqiao Hotel
26,Bambous Virieux,MU,70.05,light rain,-20.3428,57.7575,Casa Tia Villa
27,Prieska,ZA,60.24,clear sky,-29.6641,22.7474,Avenue GuestHouse
32,Geraldton,AU,60.93,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
39,Asmara,ER,61.95,broken clouds,15.3333,38.9333,Hamasien Hotel
41,Novaya Ladoga,RU,64.89,clear sky,60.1025,32.3019,База Отдыха Горизонт
46,Inhambane,MZ,64.38,few clouds,-23.8650,35.3833,Oceano
56,Roald,NO,60.17,light intensity drizzle,62.5833,6.1333,Flem Cabins


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))